In [1]:
import os
import sys
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
pd.options.display.max_columns = None
pd.options.display.max_rows = None
from modules import *
from execution import *
import json

In [2]:
gpt2 = {'l': 1024, 'e': 1600, 'h': 32, 'depth': 48}
gpt3 = {'l': 2048, 'e': 12288, 'h': 256, 'depth': 96}
gpt3_lowdepth = {'l': 2048, 'e': 12288, 'h': 256, 'depth': 96 // 8}
vit_era5 = {'l': 64800, 'e': 5120, 'h': 256, 'depth': 24}
### model
model = gpt3
global_batch = 1024
l = model['l']
e = model['e']
f = 4 * e
model['f'] = f
h = model['h']
depth = model['depth']
print(model)

{'l': 2048, 'e': 12288, 'h': 256, 'depth': 96, 'f': 49152}


In [3]:
with open('config.json', 'r') as file:
    system = json.load(file)

In [4]:
def print_df(df_mlp, df_sa):
    cols = df_mlp.columns.tolist()
    layer_track_cols = ['activation_buffer', 'weights_mem', 
                        'weights_grad_mem', 'flops_fwd', 'flops_bwd', 
                        't_fwd', 't_fwd_comm', 't_bwd', 't_bwd_comm']
    display(df_mlp[cols])
    display(df_mlp[layer_track_cols].sum())
    display(df_sa[cols])
    display(df_sa[layer_track_cols].sum()) 

In [5]:
system['nvlink_size'] = 4

In [6]:
m = 16 # total model parallel devices
pp = 2
tp = m // pp
dp = 128
b = 1
m1 = tp
t1 = 'nvlink' if m1 <= system['nvlink_size'] else 'ib'
df_mlp = mlp_1d(b, l, e, f, depth, parallelism={'m': m1}, topology={'t': t1})
df_sa = sa_1d(b, l, e, h, depth, parallelism={'m': m1}, topology={'t': t1}, flash_attention=True)
print_df(df_mlp, df_sa)
_,_ = totals(df_mlp, df_sa, model['depth'], pp=pp, dp=1, verbose=True)

,name,weights_mem,weights_grad_mem,flops_fwd,activation_buffer,comm_fwd,comm_fwd_type,flops_bwd,comm_bwd,comm_bwd_type,t_fwd,t_fwd_comm,intensity_fwd,t_bwd,t_bwd_comm,intensity_bwd,t
0,fc1,0.150995,0.150995,0.309225,0.050332,0.000000,reducescatter,0.618375,0.050332,reducescatter,0.991106,0.000000,6.804509,2.485286,0.503316,6.803679,3.476392
1,fc1-bias,0.000012,0.000012,0.000013,0.000000,0.000000,none,0.000013,0.000000,allreduce,0.016192,0.000000,0.009963,0.016192,0.000000,0.009963,0.032383
2,act1,0.000000,0.000000,0.000013,0.025166,0.000000,none,0.000013,0.000000,none,0.032368,0.000000,0.004984,0.048551,0.000000,0.003323,0.080919
3,dpr1,0.000000,0.000000,0.000013,0.012583,0.000000,none,0.000013,0.000000,none,0.040460,0.000000,0.003987,0.040460,0.000000,0.003987,0.080919
4,fc2,0.150995,0.150995,0.309212,0.025166,0.050332,reducescatter,0.618387,0.000000,reducescatter,1.494382,0.503316,6.804233,1.982010,0.000000,6.803817,3.476392
5,fc2-bias,0.000025,0.000025,0.000025,0.000000,0.000000,none,0.000025,0.000000,allreduce,0.032383,0.000000,0.009963,0.032383,0.000000,0.009963,0.064767
6,dpr2,0.000000,0.000000,0.000003,0.003146,0.000000,none,0.000003,0.000000,none,0.010115,0.000000,0.003987,0.010115,0.000000,0.003987,0.020230
7,ln1,0.000049,0.000049,0.000028,0.006291,0.050332,allgather,0.000038,0.050332,reducescatter,0.511441,0.503316,0.044678,0.523578,0.503316,0.023885,1.035019


activation_buffer    0.122683
weights_mem          0.302076
weights_grad_mem     0.302076
flops_fwd            0.618532
flops_bwd            1.236866
t_fwd                3.128446
t_fwd_comm           1.006633
t_bwd                5.138576
t_bwd_comm           1.006633
dtype: float64

,name,weights_mem,weights_grad_mem,flops_fwd,activation_buffer,comm_fwd,comm_fwd_type,flops_bwd,comm_bwd,comm_bwd_type,t_fwd,t_fwd_comm,intensity_fwd,t_bwd,t_bwd_comm,intensity_bwd,t
0,qkv,0.113246,0.113246,0.231919,0.050332,0.000000,reducescatter,0.463775,0.050332,reducescatter,0.743329,0.000000,6.335233,1.989774,0.503316,6.334374,2.733103
1,fusedla,0.000000,0.000000,0.026169,0.025428,0.000000,none,0.065354,0.000000,none,0.335503,0.000000,20.623376,0.837878,0.000000,25.752170,1.173381
2,vproj,0.037749,0.037749,0.077284,0.006291,0.050332,reducescatter,0.154597,0.000000,reducescatter,0.751022,0.503316,4.081543,0.495503,0.000000,4.082290,1.246525
3,vproj-bias,0.000025,0.000025,0.000025,0.000000,0.000000,none,0.000025,0.000000,allreduce,0.032383,0.000000,0.009963,0.032383,0.000000,0.009963,0.064767
4,dpr_v,0.000000,0.000000,0.000003,0.003146,0.000000,none,0.000003,0.000000,none,0.010115,0.000000,0.003987,0.010115,0.000000,0.003987,0.020230
5,ln2,0.000049,0.000049,0.000028,0.006291,0.050332,allgather,0.000038,0.050332,reducescatter,0.511441,0.503316,0.044678,0.523578,0.503316,0.023885,1.035019


activation_buffer    0.091488
weights_mem          0.151069
weights_grad_mem     0.151069
flops_fwd            0.335429
flops_bwd            0.683792
t_fwd                2.383794
t_fwd_comm           1.006633
t_bwd                3.889230
t_bwd_comm           1.006633
dtype: float64

time for 1 itr = 1395.844387437036
mem consumed = 179.031178752


In [7]:
n_gpus = [1, 4, 16, 36, 64]
with open('config.json', 'r') as file:
    system = json.load(file)
system['nvlink_size'] = 4
configs = execute_1d(model, n_gpus, dp=256, microbatch=1, system=system)
print(configs)

NameError: name 'l' is not defined